In [263]:
import numpy as np
import dgl
import torch
from pymatgen.core.structure import Structure
import pandas as pd
# import cugraph
# import cudf
from dgl import backend as F
from copy import deepcopy
import matplotlib.pyplot as plt
from pymatgen.electronic_structure.core import Orbital
from pymatgen.io.cif import CifWriter 
import scipy.constants as C

In [239]:
cif_file = '/media/pigz2538/e30413cf-5429-46b3-8fe9-ea458052c437/文档/ptbmodel/datas/test_data/raw/MoS2_p6m2/MoS2_p6m2.cif'
crystal = Structure.from_file(cif_file)
crystaltrans = deepcopy(crystal)
crystal.to_unit_cell = True

1


/home/pigz2538/Tools/anaconda3/envs/matformer/lib/python3.10/site-packages/pymatgen/io/cif.py:1200: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [247]:
crystal.rotate_sites([0,1,2],theta=(np.pi/2),axis=np.array([0,0,1]),anchor=[0.5,0.5,0.5], to_unit_cell=False)

In [241]:
crystaltrans.cart_coords

array([[ 0.        ,  0.        ,  9.06355632],
       [ 1.59203323,  0.91916082, 10.62711265],
       [ 1.59203323,  0.91916082,  7.5       ]])

In [248]:
crystal.cart_coords

array([[-2.22044605e-16,  5.55111512e-17,  9.06355632e+00],
       [ 1.59203323e+00,  9.19160815e-01,  1.06271127e+01],
       [ 1.59203323e+00,  9.19160815e-01,  7.50000000e+00]])

In [249]:
crystal.to_unit_cell = True

In [250]:
crystal.cart_coords

array([[-2.22044605e-16,  5.55111512e-17,  9.06355632e+00],
       [ 1.59203323e+00,  9.19160815e-01,  1.06271127e+01],
       [ 1.59203323e+00,  9.19160815e-01,  7.50000000e+00]])

In [1]:
import os
import shutil
import json

import numpy as np
import dgl
import torch
from pymatgen.core.structure import Structure
import pandas as pd
# import cugraph
# import cudf
from dgl import backend as F
from copy import deepcopy
import matplotlib.pyplot as plt
from pymatgen.electronic_structure.core import Orbital
from pymatgen.io.cif import CifWriter 
import scipy.constants as C

In [2]:
cif_files = '/media/pigz2538/e30413cf-5429-46b3-8fe9-ea458052c437/文档/cifspin/ciffiles'
cif_files_rotate = '/media/pigz2538/e30413cf-5429-46b3-8fe9-ea458052c437/文档/cifspin/spinall'

In [3]:
def spincif(original, aim):
    if not os.path.exists(aim):
        os.makedirs(aim)

    rad2theta = {'30': np.pi / 6, '60': np.pi / 3, '90': np.pi / 2, 
                 '120': np.pi * 2 / 3, '150': np.pi * 5 / 6, '180': np.pi, 
                 '210': np.pi * 7 / 6, '240': np.pi * 4 / 3, '270': np.pi * 2 / 3,
                 '300': np.pi * 5 / 3, '330': np.pi * 11 / 6}
    axises = {'x': np.array([1,0,0])}

    for theta in rad2theta.keys():
        for axis in axises.keys():
            for root, dirs, files in os.walk(original):
                for file in files:
                # 选取后缀名为.cif的文件
                    if file.endswith(".cif"):
                        crystal = Structure.from_file(os.path.join(original, file))
                        crystal.rotate_sites([0,1,2],theta=rad2theta[theta],axis=axises[axis],anchor=[0.5,0.5,0.5], to_unit_cell=False)
                        crystal.to_file(os.path.join(aim, file[:-4] + '_' + axis + '_' + theta + '.cif'))

In [4]:
spincif(cif_files, cif_files_rotate)

/home/pigz2538/Tools/anaconda3/envs/matformer/lib/python3.10/site-packages/pymatgen/io/cif.py:1199: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [5]:
data_path = '/media/pigz2538/e30413cf-5429-46b3-8fe9-ea458052c437/文档/cifspin/original'
cifspin_path = '/media/pigz2538/e30413cf-5429-46b3-8fe9-ea458052c437/文档/cifspin/spinall'
spin_data_path = '/media/pigz2538/e30413cf-5429-46b3-8fe9-ea458052c437/文档/cifspin/spindata'

for root, dirs, files in os.walk(cifspin_path):
    for file in files:
    # 选取后缀名为.cif的文件
        if file.endswith(".cif"):
            name = file.split('_')[0] + '_' + file.split('_')[1]
            axis = file.split('_')[2]
            angle = file.split('_')[3].replace('.cif','')

            folder = os.path.join(spin_data_path, name + '_' + axis + '_' + angle)

            if not os.path.exists(folder):
                os.makedirs(folder)
            
            shutil.copy2(os.path.join(cifspin_path, file), os.path.join(folder, file))

            find_path = os.path.join(data_path, name)
            for root2, dirs2, files2 in os.walk(find_path):
                for file2 in files2:
                    if file2.endswith(".cif") or file2.endswith(".txt"):
                        continue
                    
                    elif file2.endswith(".json"):
                        with open(os.path.join(find_path, file2), encoding="utf-8") as f:  # 注意编码要和文件编码一致，不加encoding参数默认使用gbk编码读取文件
                            config = json.load(f)
                        
                        config["filename"] = config["filename"].replace('.py','').replace('.txt','').replace('.cif','') + '_' + axis + '_' + angle

                        with open(os.path.join(folder, config["filename"] + '.json'), "w") as w:
                            json.dump(config, w, ensure_ascii=False)
                    else:
                        shutil.copy2(os.path.join(find_path, file2), os.path.join(folder, file2))